#### Data Partitioning

* Dataset is to be partitioned into three parts
1. TRAIN dataset - contains 80% to 95% of the whole data depending on the number of images and it is used to train the Neural network
* Metrics include: training loss and training accuracy
2. DEV (validation) dataset: Contains 5% to 2.5% of the whole data depending on the number of images
* Used to validate the neural network model on unseen data during the training
* Metrics include: validation loss and validation accuracy
3. TEST dataset
* Contains 5% to 2.5% of the whole data depending on the number of images
* Used to test the neural network model on unseen data after completion of the training
* Metrics: Test Loss and Test Accuracy

#### Loading data from HDF5 Dataset file

In [2]:
from pathlib import Path
IMAGE_SIZE = 224


In [3]:
# Change the paths below according to the system on which the code is being run
# good to have the size in the name in case you create datasets with different image sizes.
DATASET_PATH = "../../dataset"
H5DATASET_FOLDER = "../../h5-dataset"  # created at run time
TARGET_HDF5_DATASET_PATH = f"{H5DATASET_FOLDER}/Vehicle-type-dataset-SIZE{IMAGE_SIZE}.hdf5"  # created when the code is run
OUTPUT_STATS_FILE = f"{H5DATASET_FOLDER}/vehicle-type-dataset-SIZE{IMAGE_SIZE}.hdf5.csv"

print(DATASET_PATH)
print(TARGET_HDF5_DATASET_PATH)

../../dataset
../../h5-dataset/Vehicle-type-dataset-SIZE224.hdf5
